# Chapter 22: Routing and Templates

**Web Development Fundamentals**

Building on the WSGI foundation from the previous notebook, this notebook explores
how web frameworks map URLs to handler functions (routing), extract parameters from
paths and query strings, render dynamic HTML with templates, and handle form
submissions. All examples use only the Python standard library to illustrate the
patterns that frameworks like Flask and Django implement under the hood.

## URL Routing: Mapping Paths to Handlers

Routing is the process of matching an incoming URL path to a handler function.
The simplest approach is a dictionary mapping path strings to callables. More
advanced routers use regex patterns to extract path parameters.

In [ ]:
from typing import Callable, Iterable
from http import HTTPStatus
import json


Environ = dict[str, object]
StartResponse = Callable[[str, list[tuple[str, str]]], Callable[..., object]]
Handler = Callable[[Environ, StartResponse], Iterable[bytes]]


class SimpleRouter:
    """A basic router that maps exact paths to handler functions."""

    def __init__(self) -> None:
        self._routes: dict[str, Handler] = {}

    def add_route(self, path: str, handler: Handler) -> None:
        """Register a handler for an exact path."""
        self._routes[path] = handler

    def resolve(self, path: str) -> Handler | None:
        """Find a handler for the given path."""
        return self._routes.get(path)

    def __call__(
        self, environ: Environ, start_response: StartResponse
    ) -> Iterable[bytes]:
        """WSGI application interface."""
        path = str(environ.get("PATH_INFO", "/"))
        handler = self.resolve(path)

        if handler is None:
            body = json.dumps({"error": "Not Found"}).encode("utf-8")
            start_response(
                f"{HTTPStatus.NOT_FOUND.value} {HTTPStatus.NOT_FOUND.phrase}",
                [("Content-Type", "application/json"),
                 ("Content-Length", str(len(body)))],
            )
            return [body]

        return handler(environ, start_response)


# Define handlers
def home_handler(environ: Environ, start_response: StartResponse) -> Iterable[bytes]:
    body = b"Welcome to the home page!"
    start_response("200 OK", [
        ("Content-Type", "text/plain"),
        ("Content-Length", str(len(body))),
    ])
    return [body]


def about_handler(environ: Environ, start_response: StartResponse) -> Iterable[bytes]:
    body = b"About this application."
    start_response("200 OK", [
        ("Content-Type", "text/plain"),
        ("Content-Length", str(len(body))),
    ])
    return [body]


# Set up the router
router = SimpleRouter()
router.add_route("/", home_handler)
router.add_route("/about", about_handler)


# Test helper
def test_route(app: SimpleRouter, path: str) -> None:
    """Simulate a GET request and print the result."""
    result_status = ""

    def start_response(status: str, headers: list[tuple[str, str]]) -> Callable[..., object]:
        nonlocal result_status
        result_status = status
        return lambda *a: None

    environ: Environ = {"REQUEST_METHOD": "GET", "PATH_INFO": path}
    body = b"".join(app(environ, start_response))
    print(f"GET {path:20s} -> {result_status:15s} | {body.decode()}")


test_route(router, "/")
test_route(router, "/about")
test_route(router, "/missing")

## Route Decorator Pattern

Frameworks like Flask use a `@app.route("/path")` decorator to register handlers.
This pattern is syntactic sugar over `add_route()` -- the decorator registers
the function in the routing table and returns it unchanged.

In [ ]:
from typing import Callable, Iterable
from http import HTTPStatus
import json


Environ = dict[str, object]
StartResponse = Callable[[str, list[tuple[str, str]]], Callable[..., object]]
Handler = Callable[[Environ, StartResponse], Iterable[bytes]]


class App:
    """A mini web framework with Flask-style route decorators."""

    def __init__(self) -> None:
        self._routes: dict[str, dict[str, Handler]] = {}  # path -> {method -> handler}

    def route(
        self, path: str, methods: list[str] | None = None
    ) -> Callable[[Handler], Handler]:
        """Decorator to register a route handler.

        Usage:
            @app.route("/users", methods=["GET", "POST"])
            def users_handler(environ, start_response):
                ...
        """
        if methods is None:
            methods = ["GET"]

        def decorator(func: Handler) -> Handler:
            if path not in self._routes:
                self._routes[path] = {}
            for method in methods:
                self._routes[path][method.upper()] = func
            return func

        return decorator

    def __call__(
        self, environ: Environ, start_response: StartResponse
    ) -> Iterable[bytes]:
        """WSGI interface: dispatch to the matching handler."""
        path = str(environ.get("PATH_INFO", "/"))
        method = str(environ.get("REQUEST_METHOD", "GET"))

        route_methods = self._routes.get(path)
        if route_methods is None:
            return self._error(HTTPStatus.NOT_FOUND, start_response)

        handler = route_methods.get(method)
        if handler is None:
            allowed = ", ".join(sorted(route_methods.keys()))
            body = json.dumps({"error": "Method Not Allowed", "allowed": allowed}).encode()
            start_response(
                f"{HTTPStatus.METHOD_NOT_ALLOWED.value} {HTTPStatus.METHOD_NOT_ALLOWED.phrase}",
                [("Content-Type", "application/json"),
                 ("Content-Length", str(len(body))),
                 ("Allow", allowed)],
            )
            return [body]

        return handler(environ, start_response)

    @staticmethod
    def _error(
        status: HTTPStatus, start_response: StartResponse
    ) -> Iterable[bytes]:
        body = json.dumps({"error": status.phrase}).encode()
        start_response(
            f"{status.value} {status.phrase}",
            [("Content-Type", "application/json"),
             ("Content-Length", str(len(body)))],
        )
        return [body]


# Create the app and register routes using decorators
app = App()


@app.route("/")
def index(environ: Environ, start_response: StartResponse) -> Iterable[bytes]:
    body = b"Home page"
    start_response("200 OK", [("Content-Type", "text/plain"),
                               ("Content-Length", str(len(body)))])
    return [body]


@app.route("/items", methods=["GET", "POST"])
def items(environ: Environ, start_response: StartResponse) -> Iterable[bytes]:
    method = str(environ.get("REQUEST_METHOD"))
    body = f"Items endpoint: {method}".encode()
    start_response("200 OK", [("Content-Type", "text/plain"),
                               ("Content-Length", str(len(body)))])
    return [body]


# Test the decorator-based routing
def test_app(app: App, method: str, path: str) -> None:
    result_status = ""

    def start_response(status: str, headers: list[tuple[str, str]]) -> Callable[..., object]:
        nonlocal result_status
        result_status = status
        return lambda *a: None

    environ: Environ = {"REQUEST_METHOD": method, "PATH_INFO": path}
    body = b"".join(app(environ, start_response))
    print(f"{method:6s} {path:15s} -> {result_status:30s} | {body.decode()}")


test_app(app, "GET", "/")
test_app(app, "GET", "/items")
test_app(app, "POST", "/items")
test_app(app, "DELETE", "/items")   # Method not allowed
test_app(app, "GET", "/unknown")    # Not found

## Path Parameters: Extracting Values from URLs

Dynamic routes like `/users/42` or `/posts/hello-world` require extracting
variable segments from the URL path. Frameworks use regex patterns (or custom
parsers) to match and capture these segments. Below we build a regex-based
router that supports `<name>` placeholders in route patterns.

In [ ]:
import re
from typing import Callable, Iterable
from dataclasses import dataclass, field
from http import HTTPStatus
import json


Environ = dict[str, object]
StartResponse = Callable[[str, list[tuple[str, str]]], Callable[..., object]]

# Handlers now receive path params as a dict
ParamHandler = Callable[[Environ, StartResponse, dict[str, str]], Iterable[bytes]]


@dataclass
class Route:
    """A route with a regex pattern compiled from a path template."""
    template: str                 # Original template, e.g., "/users/<user_id>"
    pattern: re.Pattern[str]      # Compiled regex
    handler: ParamHandler
    methods: set[str] = field(default_factory=lambda: {"GET"})


def compile_route_pattern(template: str) -> re.Pattern[str]:
    """Convert a route template like '/users/<user_id>' to a regex.

    <name> captures a path segment as a named group matching [^/]+.
    """
    # Replace <name> with named regex groups
    regex = re.sub(r"<(\w+)>", r"(?P<\1>[^/]+)", template)
    return re.compile(f"^{regex}$")


class RegexRouter:
    """A router that supports path parameters via regex."""

    def __init__(self) -> None:
        self._routes: list[Route] = []

    def route(
        self, template: str, methods: list[str] | None = None
    ) -> Callable[[ParamHandler], ParamHandler]:
        if methods is None:
            methods = ["GET"]

        def decorator(func: ParamHandler) -> ParamHandler:
            route = Route(
                template=template,
                pattern=compile_route_pattern(template),
                handler=func,
                methods=set(m.upper() for m in methods),
            )
            self._routes.append(route)
            return func

        return decorator

    def __call__(
        self, environ: Environ, start_response: StartResponse
    ) -> Iterable[bytes]:
        path = str(environ.get("PATH_INFO", "/"))
        method = str(environ.get("REQUEST_METHOD", "GET"))

        for route in self._routes:
            match = route.pattern.match(path)
            if match:
                if method not in route.methods:
                    body = json.dumps({"error": "Method Not Allowed"}).encode()
                    start_response("405 Method Not Allowed", [
                        ("Content-Type", "application/json"),
                        ("Content-Length", str(len(body))),
                    ])
                    return [body]
                params = match.groupdict()
                return route.handler(environ, start_response, params)

        body = json.dumps({"error": "Not Found"}).encode()
        start_response("404 Not Found", [
            ("Content-Type", "application/json"),
            ("Content-Length", str(len(body))),
        ])
        return [body]


# Demonstrate regex-based routing
print("=== Pattern Compilation ===")
test_templates = ["/users/<user_id>", "/posts/<slug>/comments/<comment_id>"]
for tmpl in test_templates:
    pattern = compile_route_pattern(tmpl)
    print(f"  {tmpl:45s} -> {pattern.pattern}")

# Build an app with parameterized routes
api = RegexRouter()


@api.route("/users/<user_id>")
def get_user(
    environ: Environ, start_response: StartResponse, params: dict[str, str]
) -> Iterable[bytes]:
    body = json.dumps({"user_id": params["user_id"], "name": "Alice"}).encode()
    start_response("200 OK", [
        ("Content-Type", "application/json"),
        ("Content-Length", str(len(body))),
    ])
    return [body]


@api.route("/posts/<slug>/comments/<comment_id>")
def get_comment(
    environ: Environ, start_response: StartResponse, params: dict[str, str]
) -> Iterable[bytes]:
    body = json.dumps({
        "post_slug": params["slug"],
        "comment_id": params["comment_id"],
    }).encode()
    start_response("200 OK", [
        ("Content-Type", "application/json"),
        ("Content-Length", str(len(body))),
    ])
    return [body]


# Test parameterized routing
print("\n=== Parameterized Route Tests ===")
for path in ["/users/42", "/users/alice", "/posts/hello-world/comments/7", "/nope"]:
    captured_status = ""

    def sr(status: str, headers: list[tuple[str, str]]) -> Callable[..., object]:
        nonlocal captured_status
        captured_status = status
        return lambda *a: None

    environ: Environ = {"REQUEST_METHOD": "GET", "PATH_INFO": path}
    body = b"".join(api(environ, sr))
    print(f"  GET {path:45s} -> {captured_status:15s} | {body.decode()}")

## Query Parameter Parsing

Query parameters appear after `?` in a URL (e.g., `/search?q=python&page=2`).
The `urllib.parse` module provides `parse_qs()` and `parse_qsl()` for parsing
these into dictionaries or lists of pairs. Note that a key can appear multiple
times, so `parse_qs()` returns lists of values.

In [ ]:
from urllib.parse import parse_qs, parse_qsl, urlencode, quote, unquote


# Basic query string parsing
query_string = "q=python+web&page=2&tag=wsgi&tag=http&sort=relevance"

print("=== parse_qs (dict of lists) ===")
params: dict[str, list[str]] = parse_qs(query_string)
for key, values in params.items():
    print(f"  {key}: {values}")

print("\n=== parse_qsl (list of pairs) ===")
pairs: list[tuple[str, str]] = parse_qsl(query_string)
for key, value in pairs:
    print(f"  {key} = {value}")

# URL encoding and decoding
print("\n=== URL Encoding ===")
raw_text = "hello world & special=chars/here"
encoded = quote(raw_text)
decoded = unquote(encoded)
print(f"  Raw:     {raw_text}")
print(f"  Encoded: {encoded}")
print(f"  Decoded: {decoded}")

# Build a query string from parameters
print("\n=== Building Query Strings ===")
search_params: dict[str, str | list[str]] = {
    "q": "python web",
    "page": "1",
    "tags": ["wsgi", "http", "rest"],  # type: ignore[dict-item]
}
# urlencode with doseq=True handles list values
qs = urlencode(search_params, doseq=True)
print(f"  Encoded: {qs}")


# Helper function for extracting typed query params
def get_query_param(
    query_string: str,
    key: str,
    default: str = "",
    as_int: bool = False,
) -> str | int:
    """Extract a single query parameter with optional int conversion."""
    params = parse_qs(query_string)
    values = params.get(key, [])
    value = values[0] if values else default
    if as_int:
        try:
            return int(value)
        except (ValueError, TypeError):
            return 0
    return value


print("\n=== Typed Query Param Extraction ===")
qs_example = "page=3&limit=25&search=python"
print(f"  page (int):  {get_query_param(qs_example, 'page', as_int=True)}")
print(f"  limit (int): {get_query_param(qs_example, 'limit', as_int=True)}")
print(f"  search (str): {get_query_param(qs_example, 'search')}")
print(f"  missing (default): {get_query_param(qs_example, 'missing', 'fallback')}")

## Template Rendering with `string.Template`

Before reaching for Jinja2 or Mako, Python's standard library offers
`string.Template` for simple variable substitution in templates. It uses
`$name` or `${name}` placeholders and is safe against code injection
(unlike f-strings, which execute arbitrary expressions).

In [ ]:
from string import Template


# Basic string.Template usage
page_template = Template("""\
<!DOCTYPE html>
<html>
<head><title>$title</title></head>
<body>
  <h1>$heading</h1>
  <p>Welcome, $username!</p>
  <p>You have $message_count new messages.</p>
</body>
</html>""")

# Substitute values into the template
rendered = page_template.substitute(
    title="Dashboard",
    heading="User Dashboard",
    username="Alice",
    message_count="5",
)
print("=== Rendered Template ===")
print(rendered)

# safe_substitute does not raise on missing keys
print("\n=== Safe Substitute (missing keys) ===")
partial = Template("Hello, $name! Your role is $role.")
result = partial.safe_substitute(name="Bob")
print(f"  Result: {result}")
print("  Note: $role was left as-is because it was missing")

# Template with ${braced} syntax for adjacent text
print("\n=== Braced Syntax ===")
braced = Template("File: ${filename}_v${version}.txt")
print(f"  {braced.substitute(filename='report', version='2')}")

## HTML Escaping for Security

Injecting user-provided data into HTML without escaping creates **Cross-Site
Scripting (XSS)** vulnerabilities. The `html.escape()` function converts
dangerous characters (`<`, `>`, `&`, `"`, `'`) into HTML entities. Always
escape user input before inserting it into HTML templates.

In [ ]:
import html
from string import Template


# Demonstrate the XSS problem
malicious_input = '<script>alert("XSS attack!")</script>'

print("=== Without Escaping (DANGEROUS) ===")
unsafe_template = Template("<p>Welcome, $username!</p>")
unsafe_html = unsafe_template.substitute(username=malicious_input)
print(f"  {unsafe_html}")
print("  ^ The script tag would execute in a browser!")

print("\n=== With html.escape() (SAFE) ===")
safe_username = html.escape(malicious_input)
safe_html = unsafe_template.substitute(username=safe_username)
print(f"  {safe_html}")
print("  ^ The script tag is rendered as harmless text.")

# What html.escape does to each character
print("\n=== Character Escaping ===")
test_chars: list[str] = ["<", ">", "&", '"', "'", "normal text"]
for char in test_chars:
    escaped = html.escape(char, quote=True)
    print(f"  {char!r:20s} -> {escaped!r}")


# A safe template rendering function
def render_template(template_str: str, **kwargs: str) -> str:
    """Render a template with all values HTML-escaped."""
    escaped_kwargs = {key: html.escape(str(value)) for key, value in kwargs.items()}
    return Template(template_str).substitute(escaped_kwargs)


print("\n=== Safe Rendering Function ===")
result = render_template(
    "<p>Hello, $name! Search for: $query</p>",
    name="Alice <admin>",
    query='test" onclick="alert(1)',
)
print(f"  {result}")

## A Template Engine with Loops and Conditionals

`string.Template` only supports simple substitution. For rendering lists and
conditional content, we need something more. Below is a minimal template engine
that uses Python string formatting with helper functions to generate HTML
from data structures -- a pattern common in server-side rendering.

In [ ]:
import html
from dataclasses import dataclass


@dataclass
class User:
    name: str
    email: str
    is_admin: bool = False


def render_user_list(users: list[User], title: str = "User List") -> str:
    """Render an HTML page listing users, with escaping."""
    rows: list[str] = []
    for user in users:
        name = html.escape(user.name)
        email = html.escape(user.email)
        badge = ' <span class="badge">Admin</span>' if user.is_admin else ""
        rows.append(
            f"    <tr><td>{name}{badge}</td><td>{email}</td></tr>"
        )

    table_body = "\n".join(rows) if rows else "    <tr><td colspan='2'>No users</td></tr>"
    safe_title = html.escape(title)

    return f"""<!DOCTYPE html>
<html>
<head><title>{safe_title}</title></head>
<body>
  <h1>{safe_title}</h1>
  <p>Total: {len(users)} user(s)</p>
  <table border="1">
    <tr><th>Name</th><th>Email</th></tr>
{table_body}
  </table>
</body>
</html>"""


# Render with normal data
users = [
    User("Alice", "alice@example.com", is_admin=True),
    User("Bob", "bob@example.com"),
    User("Charlie <script>alert('xss')</script>", "charlie@example.com"),
]

print(render_user_list(users, "Team Members"))
print("\n--- Note: Charlie's malicious name is safely escaped. ---")

## Form Handling: Parsing POST Data

HTML forms submit data as URL-encoded key-value pairs in the request body
(Content-Type: `application/x-www-form-urlencoded`). The `urllib.parse.parse_qs()`
function parses this format, just as it does query strings. For file uploads,
forms use `multipart/form-data`, which requires more complex parsing.

In [ ]:
import io
from urllib.parse import parse_qs, urlencode
from typing import Callable, Iterable
import json
import html


Environ = dict[str, object]
StartResponse = Callable[[str, list[tuple[str, str]]], Callable[..., object]]


def parse_form_data(environ: Environ) -> dict[str, list[str]]:
    """Parse URL-encoded form data from a POST request body."""
    content_type = str(environ.get("CONTENT_TYPE", ""))

    if "application/x-www-form-urlencoded" not in content_type:
        return {}

    try:
        content_length = int(str(environ.get("CONTENT_LENGTH", "0") or "0"))
    except ValueError:
        content_length = 0

    wsgi_input = environ.get("wsgi.input")
    if not wsgi_input or content_length == 0:
        return {}

    body = wsgi_input.read(content_length)  # type: ignore[union-attr]
    return parse_qs(body.decode("utf-8"))


# Simulate a form submission
form_body = urlencode({
    "username": "alice",
    "email": "alice@example.com",
    "role": "admin",
}).encode("utf-8")

form_environ: Environ = {
    "REQUEST_METHOD": "POST",
    "PATH_INFO": "/register",
    "CONTENT_TYPE": "application/x-www-form-urlencoded",
    "CONTENT_LENGTH": str(len(form_body)),
    "wsgi.input": io.BytesIO(form_body),
}

print("=== Raw Form Body ===")
print(f"  {form_body.decode()}")

print("\n=== Parsed Form Data ===")
form_data = parse_form_data(form_environ)
for key, values in form_data.items():
    print(f"  {key}: {values}")


# A WSGI handler that renders a form and processes submissions
def form_handler(
    environ: Environ, start_response: StartResponse
) -> Iterable[bytes]:
    method = str(environ.get("REQUEST_METHOD", "GET"))

    if method == "GET":
        # Render the form
        form_html = """<!DOCTYPE html>
<html><body>
  <h1>Registration</h1>
  <form method="POST">
    <label>Username: <input name="username"></label><br>
    <label>Email: <input name="email" type="email"></label><br>
    <button type="submit">Register</button>
  </form>
</body></html>"""
        body = form_html.encode("utf-8")
        start_response("200 OK", [
            ("Content-Type", "text/html; charset=utf-8"),
            ("Content-Length", str(len(body))),
        ])
        return [body]

    elif method == "POST":
        # Process the form data
        data = parse_form_data(environ)
        username = html.escape(data.get("username", [""])[0])
        email = html.escape(data.get("email", [""])[0])

        response_html = f"""<!DOCTYPE html>
<html><body>
  <h1>Registration Successful</h1>
  <p>Username: {username}</p>
  <p>Email: {email}</p>
</body></html>"""
        body = response_html.encode("utf-8")
        start_response("200 OK", [
            ("Content-Type", "text/html; charset=utf-8"),
            ("Content-Length", str(len(body))),
        ])
        return [body]

    else:
        body = b"Method Not Allowed"
        start_response("405 Method Not Allowed", [
            ("Content-Type", "text/plain"),
            ("Content-Length", str(len(body))),
        ])
        return [body]


# Test GET (show form)
print("\n=== GET /register (show form) ===")
status_result = ""
def sr(s: str, h: list[tuple[str, str]]) -> Callable[..., object]:
    global status_result
    status_result = s
    return lambda *a: None

body_out = b"".join(form_handler({"REQUEST_METHOD": "GET", "PATH_INFO": "/register"}, sr))
print(f"  Status: {status_result}")
print(f"  Contains <form>: {'<form' in body_out.decode()}")

# Test POST (process form)
print("\n=== POST /register (process form) ===")
post_body = b"username=alice&email=alice%40example.com"
body_out = b"".join(form_handler({
    "REQUEST_METHOD": "POST",
    "PATH_INFO": "/register",
    "CONTENT_TYPE": "application/x-www-form-urlencoded",
    "CONTENT_LENGTH": str(len(post_body)),
    "wsgi.input": io.BytesIO(post_body),
}, sr))
print(f"  Status: {status_result}")
print(f"  Contains 'Registration Successful': {'Registration Successful' in body_out.decode()}")

## Multipart Form Data Concepts

File uploads use `multipart/form-data` encoding, where the body is split into
parts separated by a boundary string. Each part has its own headers (including
`Content-Disposition` with the field name and filename). Python's `cgi` module
(deprecated) or `multipart` library can parse this format. Below we examine
the structure of multipart data.

In [ ]:
# Demonstrate the structure of multipart form data

boundary = "----FormBoundary7MA4YWxkTrZu0gW"

# This is what a browser sends for a form with a text field and a file upload
multipart_body = (
    f"--{boundary}\r\n"
    f'Content-Disposition: form-data; name="username"\r\n'
    f"\r\n"
    f"alice\r\n"
    f"--{boundary}\r\n"
    f'Content-Disposition: form-data; name="avatar"; filename="photo.png"\r\n'
    f"Content-Type: image/png\r\n"
    f"\r\n"
    f"<binary image data would go here>\r\n"
    f"--{boundary}--\r\n"
)

print("=== Multipart Form Data Structure ===")
print(f"Content-Type: multipart/form-data; boundary={boundary}")
print(f"Content-Length: {len(multipart_body)}")
print()
print(multipart_body)

# Parse the parts manually (for educational purposes)
print("=== Parsing Parts ===")
parts = multipart_body.split(f"--{boundary}")
for i, part in enumerate(parts):
    part = part.strip()
    if not part or part == "--":
        continue
    # Split headers from body at the blank line
    if "\r\n\r\n" in part:
        header_section, body_section = part.split("\r\n\r\n", 1)
        print(f"\nPart {i}:")
        print(f"  Headers: {header_section}")
        print(f"  Body:    {body_section[:50]}")

print("\n--- In practice, use the 'multipart' package or framework utilities. ---")

## Static File Serving Patterns

During development, Python applications often need to serve static files (CSS,
JavaScript, images). In production, a reverse proxy (nginx, Caddy) handles this.
Below is a WSGI middleware that serves static files from a directory, with
content type detection and basic security (preventing directory traversal).

In [ ]:
import mimetypes
import os
from pathlib import Path
from typing import Callable, Iterable
from http import HTTPStatus


Environ = dict[str, object]
StartResponse = Callable[[str, list[tuple[str, str]]], Callable[..., object]]
WSGIApp = Callable[[Environ, StartResponse], Iterable[bytes]]


class StaticFileMiddleware:
    """WSGI middleware that serves static files from a directory.

    Requests matching the url_prefix are served from static_dir.
    All other requests are passed through to the wrapped app.
    """

    def __init__(
        self,
        app: WSGIApp,
        static_dir: str | Path,
        url_prefix: str = "/static",
    ) -> None:
        self._app = app
        self._static_dir = Path(static_dir).resolve()
        self._url_prefix = url_prefix.rstrip("/")

    def __call__(
        self, environ: Environ, start_response: StartResponse
    ) -> Iterable[bytes]:
        path = str(environ.get("PATH_INFO", "/"))

        if not path.startswith(self._url_prefix + "/"):
            return self._app(environ, start_response)

        # Extract the file path relative to the static directory
        relative_path = path[len(self._url_prefix) + 1:]
        file_path = (self._static_dir / relative_path).resolve()

        # Security: prevent directory traversal
        if not str(file_path).startswith(str(self._static_dir)):
            body = b"Forbidden"
            start_response(
                f"{HTTPStatus.FORBIDDEN.value} {HTTPStatus.FORBIDDEN.phrase}",
                [("Content-Type", "text/plain"),
                 ("Content-Length", str(len(body)))],
            )
            return [body]

        if not file_path.is_file():
            body = b"Not Found"
            start_response(
                f"{HTTPStatus.NOT_FOUND.value} {HTTPStatus.NOT_FOUND.phrase}",
                [("Content-Type", "text/plain"),
                 ("Content-Length", str(len(body)))],
            )
            return [body]

        # Serve the file
        content_type, _ = mimetypes.guess_type(str(file_path))
        if content_type is None:
            content_type = "application/octet-stream"

        body = file_path.read_bytes()
        start_response(
            f"{HTTPStatus.OK.value} {HTTPStatus.OK.phrase}",
            [
                ("Content-Type", content_type),
                ("Content-Length", str(len(body))),
                ("Cache-Control", "public, max-age=3600"),
            ],
        )
        return [body]


# Demonstrate MIME type detection
print("=== MIME Type Detection ===")
test_files = ["style.css", "app.js", "image.png", "data.json", "page.html", "doc.pdf"]
for filename in test_files:
    mime_type, encoding = mimetypes.guess_type(filename)
    print(f"  {filename:15s} -> {mime_type or 'unknown'}")

# Demonstrate path traversal prevention
print("\n=== Directory Traversal Prevention ===")
static_root = Path("/var/www/static").resolve()
test_paths = [
    "style.css",
    "../../../etc/passwd",
    "images/../../../secret.txt",
    "valid/nested/file.js",
]
for rel_path in test_paths:
    resolved = (static_root / rel_path).resolve()
    is_safe = str(resolved).startswith(str(static_root))
    status = "SAFE" if is_safe else "BLOCKED"
    print(f"  {rel_path:35s} -> [{status}] {resolved}")

## Putting It Together: A Mini Web Application

Below we combine routing, templates, query parameters, form handling, and
static file patterns into a cohesive mini web application. This demonstrates
how frameworks organize these concerns.

In [ ]:
import re
import json
import html
import io
from http import HTTPStatus
from typing import Callable, Iterable
from urllib.parse import parse_qs
from string import Template
from dataclasses import dataclass, field


Environ = dict[str, object]
StartResponse = Callable[[str, list[tuple[str, str]]], Callable[..., object]]


# Simple in-memory "database"
USERS: dict[str, dict[str, str]] = {
    "1": {"name": "Alice", "email": "alice@example.com"},
    "2": {"name": "Bob", "email": "bob@example.com"},
}

# HTML templates
BASE_TEMPLATE = Template("""<!DOCTYPE html>
<html>
<head><title>$title</title></head>
<body>
  <nav><a href="/">Home</a> | <a href="/users">Users</a></nav>
  <hr>
  $content
</body>
</html>""")


def render_page(title: str, content: str) -> bytes:
    """Render content inside the base template."""
    page = BASE_TEMPLATE.substitute(title=html.escape(title), content=content)
    return page.encode("utf-8")


def html_response(
    body: bytes,
    start_response: StartResponse,
    status: HTTPStatus = HTTPStatus.OK,
) -> Iterable[bytes]:
    start_response(f"{status.value} {status.phrase}", [
        ("Content-Type", "text/html; charset=utf-8"),
        ("Content-Length", str(len(body))),
    ])
    return [body]


# Route handlers
def handle_home(
    environ: Environ, start_response: StartResponse, params: dict[str, str]
) -> Iterable[bytes]:
    body = render_page("Home", "<h1>Welcome!</h1><p>A mini web app built with WSGI.</p>")
    return html_response(body, start_response)


def handle_user_list(
    environ: Environ, start_response: StartResponse, params: dict[str, str]
) -> Iterable[bytes]:
    # Support optional search query parameter
    qs = str(environ.get("QUERY_STRING", ""))
    search = parse_qs(qs).get("q", [""])[0].lower()

    rows: list[str] = []
    for uid, user in USERS.items():
        if search and search not in user["name"].lower():
            continue
        name = html.escape(user["name"])
        email = html.escape(user["email"])
        rows.append(f"<tr><td><a href='/users/{uid}'>{name}</a></td><td>{email}</td></tr>")

    table = "<table border='1'><tr><th>Name</th><th>Email</th></tr>" + "".join(rows) + "</table>"
    search_form = (
        '<form method="GET"><input name="q" placeholder="Search..." '
        f'value="{html.escape(search)}"><button>Search</button></form>'
    )
    content = f"<h1>Users</h1>{search_form}{table}"
    body = render_page("Users", content)
    return html_response(body, start_response)


def handle_user_detail(
    environ: Environ, start_response: StartResponse, params: dict[str, str]
) -> Iterable[bytes]:
    user_id = params["user_id"]
    user = USERS.get(user_id)
    if user is None:
        body = render_page("Not Found", "<h1>User not found</h1>")
        return html_response(body, start_response, HTTPStatus.NOT_FOUND)

    name = html.escape(user["name"])
    email = html.escape(user["email"])
    content = f"<h1>{name}</h1><p>Email: {email}</p><a href='/users'>Back to list</a>"
    body = render_page(name, content)
    return html_response(body, start_response)


# Build the router using our RegexRouter from earlier
@dataclass
class MiniRoute:
    pattern: re.Pattern[str]
    handler: Callable[..., Iterable[bytes]]


routes: list[MiniRoute] = [
    MiniRoute(re.compile(r"^/$"), handle_home),
    MiniRoute(re.compile(r"^/users$"), handle_user_list),
    MiniRoute(re.compile(r"^/users/(?P<user_id>\w+)$"), handle_user_detail),
]


def mini_app(environ: Environ, start_response: StartResponse) -> Iterable[bytes]:
    path = str(environ.get("PATH_INFO", "/"))
    for route in routes:
        match = route.pattern.match(path)
        if match:
            return route.handler(environ, start_response, match.groupdict())

    body = render_page("Not Found", "<h1>404 - Page not found</h1>")
    return html_response(body, start_response, HTTPStatus.NOT_FOUND)


# Test the mini application
def test_mini(method: str, path: str, query_string: str = "") -> None:
    status_line = ""
    def sr(s: str, h: list[tuple[str, str]]) -> Callable[..., object]:
        nonlocal status_line
        status_line = s
        return lambda *a: None

    environ: Environ = {
        "REQUEST_METHOD": method,
        "PATH_INFO": path,
        "QUERY_STRING": query_string,
    }
    body = b"".join(mini_app(environ, sr))
    # Show just a snippet of the body
    decoded = body.decode()
    has_users_table = "<table" in decoded
    print(f"{method} {path + ('?' + query_string if query_string else ''):30s} "
          f"-> {status_line:15s} (table: {has_users_table}, len: {len(body)})")


print("=== Mini Web App Tests ===")
test_mini("GET", "/")
test_mini("GET", "/users")
test_mini("GET", "/users", "q=alice")
test_mini("GET", "/users/1")
test_mini("GET", "/users/2")
test_mini("GET", "/users/999")
test_mini("GET", "/nonexistent")

## Summary

This notebook covered the core patterns used by web frameworks for routing and
rendering:

1. **URL routing**: Mapping paths to handler functions using dictionaries and
   regex patterns
2. **Route decorators**: The `@app.route("/path")` pattern that registers
   handlers declaratively (as used by Flask)
3. **Path parameters**: Extracting dynamic values from URL segments with
   named regex groups (`<user_id>`)
4. **Query parameters**: Parsing `?key=value` pairs with `urllib.parse.parse_qs()`
5. **Template rendering**: Using `string.Template` for simple substitution and
   building HTML programmatically
6. **HTML escaping**: Preventing XSS attacks with `html.escape()` -- always
   escape user input before inserting it into HTML
7. **Form handling**: Parsing URL-encoded POST data and understanding multipart
   form structure for file uploads
8. **Static file serving**: MIME type detection, directory traversal prevention,
   and middleware patterns for serving files

The next notebook covers REST API design, JSON handling, middleware, error
handling, and CORS -- building on these routing fundamentals to create
API-focused web applications.